In [1]:
import numpy as np

In [2]:
starting_board = np.array([[0, 1, 1, 0],
[2, 1, 1, 2],
[3, 4, 5, 3],
[2, 6, 7, 2],
[3, 8, 9, 3]])

In [3]:
number_to_piece = {}
number_to_labels = {}

# 2x2 piece
# 6 7
# 8 9
number_to_piece[6] = [(0, 0), (0, 1), (1, 0), (1, 1)]
number_to_piece[7] = [(0, -1), (0, 0), (1, -1), (1, 0)]
number_to_piece[8] = [(-1, 0), (-1, 1), (0, 0), (0, 1)]
number_to_piece[9] = [(-1, -1), (-1, 0), (0, -1), (0, 0)]
for i in [6, 7, 8, 9]:
    number_to_labels[i] = [6, 7, 8, 9]


# Vertical 2x1 pieces
# 2
# 3
number_to_piece[2] = [(0, 0), (1, 0)]
number_to_piece[3] = [(-1, 0), (0, 0)]
for i in [2, 3]:
    number_to_labels[i] = [2, 3]


# Horizontal 1x2 piece
# 4 5
number_to_piece[4] = [(0, 0), (0, 1)]
number_to_piece[5] = [(0, -1), (0, 0)]
for i in [4, 5]:
    number_to_labels[i] = [4, 5]


# 1x1 pieces
# 1
number_to_piece[1] = [(0, 0)]
number_to_labels[1] = [1]

for n in number_to_piece:
    number_to_piece[n] = np.array(number_to_piece[n])

In [4]:
def get_board_zeros(board):
    board_zeros = []
    for i in range(5):
        for j in range(4):
            if board[i][j] == 0:
                board_zeros.append([i, j])
    return board_zeros

def in_board(square):
    return (square[0] >= 0) & (square[0] < 5) & (square[1] >= 0) & (square[1] < 4)

def count_zeros(board):
    num_zeros = 0
    for i in range(5):
        for j in range(4):
            if board[i][j] == 0:
                num_zeros += 1
    return num_zeros

def get_board_string(board):
    board_string = ""
    for i in range(5):
        for j in range(4):
            board_string += str(board[i][j])
    return board_string

def is_goal(board):
    return board[0][1] == 6

def get_successors(board):
    neighboring_boards = []
    
    for zero in get_board_zeros(board):
        directions = [np.array([-1, 0]), np.array([1, 0]), np.array([0, -1]), np.array([0, 1])]
        for direction in directions:
            neighboring_square = zero + direction
            if in_board(neighboring_square):
                label = board[neighboring_square[0]][neighboring_square[1]]
                if label != 0:
                    piece = neighboring_square + number_to_piece[label]
                    labels = number_to_labels[label]

                    new_board = board.copy()
                    for (i, j) in piece:
                        new_board[i][j] = 0
                    for (n, (i, j)) in enumerate(piece):
                        new_board[i-direction[0]][j-direction[1]] = labels[n]
                        
                    if count_zeros(new_board) == 2:
                        neighboring_boards.append(new_board)
    
    return neighboring_boards

In [5]:
# From https://en.wikipedia.org/wiki/Breadth-first_search

def breadth_first_search():
    open_set = [[starting_board]]
    closed_set = set([get_board_string(starting_board)])
    
    while len(open_set) > 0:
        board_path = open_set.pop(0)
        if is_goal(board_path[-1]):
            return board_path
        
        for subtree_root in get_successors(board_path[-1]):
            board_string = get_board_string(subtree_root)
            if board_string not in closed_set:
                open_set.append(board_path + [subtree_root])
                closed_set.add(board_string)

%time board_path = breadth_first_search()
print(len(board_path))

Wall time: 50.5 s
115


In [6]:
def find_number_states():
    open_set = [[starting_board]]
    closed_set = set([get_board_string(starting_board)])
    
    while len(open_set) > 0:
        board_path = open_set.pop(0)
        
        for subtree_root in get_successors(board_path[-1]):
            board_string = get_board_string(subtree_root)
            if board_string not in closed_set:
                open_set.append(board_path + [subtree_root])
                closed_set.add(board_string)
                
    return len(closed_set)

num_states = find_number_states()
num_states

25955

In [7]:
piece_to_ascii = {}

piece_to_ascii[0] = """. . . \n. . . \n. . . """
piece_to_ascii[1] = """. . . \n. # . \n. . . """
piece_to_ascii[2] = """. . . \n. # . \n. # . """
piece_to_ascii[3] = """. # . \n. # . \n. . . """
piece_to_ascii[4] = """. . . \n. # # \n. . . """
piece_to_ascii[5] = """. . . \n# # . \n. . . """
piece_to_ascii[6] = """. . . \n. # # \n. # # """
piece_to_ascii[7] = """. . . \n# # . \n# # . """
piece_to_ascii[8] = """. # # \n. # # \n. . . """
piece_to_ascii[9] = """# # . \n# # . \n. . . """

def board_to_ascii(board):
    ascii_board = ""
    for row in board:
        for i in range(3):
            for piece in row:
                piece_row =  piece_to_ascii[piece].split("\n")[i]
                for x in piece_row:
                    ascii_board += x
            ascii_board += "\n"
    return ascii_board

In [8]:
for (i,board) in enumerate(board_path):
    print(str(i) + ")")
    print(board_to_ascii(board), "\n")

0)
. . . . . . . . . . . . 
. . . . # . . # . . . . 
. . . . . . . . . . . . 
. . . . . . . . . . . . 
. # . . # . . # . . # . 
. # . . . . . . . . # . 
. # . . . . . . . . # . 
. # . . # # # # . . # . 
. . . . . . . . . . . . 
. . . . . . . . . . . . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. . . . . . . . . . . . 
 

1)
. . . . . . . . . . . . 
. # . . # . . # . . . . 
. # . . . . . . . . . . 
. # . . . . . . . . . . 
. # . . # . . # . . # . 
. . . . . . . . . . # . 
. . . . . . . . . . # . 
. . . . # # # # . . # . 
. . . . . . . . . . . . 
. . . . . . . . . . . . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. # . . # # # # . . # . 
. . . . . . . . . . . . 
 

2)
. . . . . . . . . . . . 
. # . . # . . # . . # . 
. # . . . . . . . . # . 
. # . . . . . . . . # . 
. # . . # . . # . . # . 
. . . . . . . . . . . . 
. . . . . . . . . . . . 
. . . . # # # # . . . . 
. . . . . . . . . . . . 
. . . . . 